In [1]:
import requests 
import pandas as pd
import numpy as np
import re
import sqlite3


In [2]:
r = requests.get('https://swapi.dev/api/').json()

In [3]:
#function to store all the data
def get_full_type_data(base_url):
    data =  requests.get(base_url)
    all_data = []
    data_num = 0
    for i in range(100):
        if data_num < data.json()['count']+1:
            trial=requests.get(base_url+str(i))
            if str(trial) != '<Response [404]>':
                all_data.append(trial.json())
                data_num+=1
    all_data = pd.json_normalize(all_data)
    return all_data

In [4]:
#get and store the people data
people = get_full_type_data('https://swapi.dev/api/people/')
films =get_full_type_data('https://swapi.dev/api/films/')
vehicles = get_full_type_data('https://swapi.dev/api/vehicles/')
starships =get_full_type_data('https://swapi.dev/api/starships/')
species = get_full_type_data('https://swapi.dev/api/species/')
planets = get_full_type_data('https://swapi.dev/api/planets/')


In [6]:
#Dropping the unnecessary columns
people.drop(labels= ['homeworld', 'starships', 'species'], axis=1, inplace=True)

films.drop(labels= ['planets', 'starships', 'species'], axis=1, inplace=True)

vehicles.drop(labels= ['pilots'], axis=1, inplace=True)



In [7]:
#Get the primary key from URL columns

people['id']=people['url'].apply(lambda x: x.split('/')[-2])


films['id_films']=films['url'].apply(lambda x: x.split('/')[-2])


vehicles['id_vehicles']=vehicles['url'].apply(lambda x: x.split('/')[-2])


In [8]:
people['height'] = people['height'].apply(lambda x: x.replace(',',''))
people['mass'] = people['mass'].apply(lambda x: x.replace(',',''))



In [9]:
#replace none values
people.replace(to_replace=[None, 'none', 'n/a', 'unknown'], value=np.nan, inplace=True)
vehicles.replace(to_replace=[None, 'none', 'n/a', 'unknown'], value=np.nan, inplace=True)


In [10]:
#convert the types of people
convert = {
    
    'name':str,
    'height':float,
    'mass':float,
    'hair_color':str,
    'skin_color':str,
    'eye_color':str,
    'birth_year':str,
    'gender':str,
    'edited':str,
    'created':str,
    'url':str
}
people= people.astype(convert)



In [14]:
people_films = people[['id','films']]

people_films = people_films.explode('films')



In [15]:
people_films['films']=people_films['films'].apply(lambda x: x.split('/')[-2])

In [16]:
people_films.rename(columns={'films': 'id_films'}, inplace=True)

In [17]:
people_vehicles=people [['id','vehicles']]

people_vehicles=people_vehicles.explode('vehicles')

In [18]:
people_vehicles.dropna(inplace=True)

people_vehicles['vehicles']=people_vehicles['vehicles'].apply(lambda x: x.split('/')[-2])



In [19]:
people.drop(labels= ['films', 'vehicles'], axis=1, inplace=True)

people.drop(columns=['edited', 'created'], axis=1, inplace=True)

films_people=films[['id_films', 'characters']]

films_people=films_people.explode('characters')

films_people['characters']=films_people['characters'].apply(lambda x: x.split('/')[-2])



In [21]:
films_vehicles=films[['id_films', 'vehicles']]

films_vehicles=films_vehicles.explode('vehicles')

films_vehicles['vehicles']=films_vehicles['vehicles'].apply(lambda x: x.split('/')[-2])



In [22]:
films.drop(columns=['characters', 'vehicles', 'created', 'edited'], axis=1, inplace=True)


In [23]:
vehicles.drop(columns=['films', 'created', 'edited'], axis=1, inplace=True)


In [24]:
conn = sqlite3.connect('starwars_database.db')
c = conn.cursor()

c.execute(f"CREATE TABLE IF NOT EXISTS people ({', '.join(people.columns)})")
conn.commit()
people.to_sql('people', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS people_films ({', '.join(people_films.columns)})")
conn.commit()
people_films.to_sql('people_films', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS people_vehicles ({', '.join(people_films.columns)})")
conn.commit()
people_vehicles.to_sql('people_vehicles', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS films ({', '.join(films.columns)})")
conn.commit()
films.to_sql('films', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS vehicles ({', '.join(vehicles.columns)})")
conn.commit()
vehicles.to_sql('vehicles', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS films_vehicles ({', '.join(films_vehicles.columns)})")
conn.commit()
films_vehicles.to_sql('films_vehicles', conn, if_exists='replace', index = False)



49

In [44]:
#select
pd.read_sql(
    '''
        SELECT 
            *
        FROM 
            people
        LIMIT 
            5
    ''', 
    conn
)



,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,url,id
0,Luke Skywalker,172.0,77.0,blond,fair,blue,19BBY,male,https://swapi.dev/api/people/1/,1
1,C-3PO,167.0,75.0,nan,gold,yellow,112BBY,nan,https://swapi.dev/api/people/2/,2
2,R2-D2,96.0,32.0,nan,"white, blue",red,33BBY,nan,https://swapi.dev/api/people/3/,3
3,Darth Vader,202.0,136.0,nan,white,yellow,41.9BBY,male,https://swapi.dev/api/people/4/,4
4,Leia Organa,150.0,49.0,brown,light,brown,19BBY,female,https://swapi.dev/api/people/5/,5


In [42]:
#Left join
pd.read_sql(
    '''
        SELECT 
            p.gender,
            COUNT(p.id) as nb_ch
        FROM 
            people p
        LEFT JOIN 
            people_films f ON f.id = p.id
        GROUP BY 
            p.gender
        ORDER BY 
            nb_ch DESC
        LIMIT 5
    ''', 
    conn
)



,gender,nb_ch
0,male,113
1,female,32
2,nan,14
3,hermaphrodite,3


In [37]:
#Group by
pd.read_sql(
    '''
        SELECT 
            model,
            count(*) AS number_of_vehicles
        FROM 
            vehicles
        WHERE
            manufacturer = 'Sienar Fleet Systems'
        GROUP BY
            model
        ORDER BY 
            number_of_vehicles DESC
        LIMIT 10
    ''', 
    conn
)


,model,number_of_vehicles
0,Twin Ion Engine/Ln Starfighter,1
1,Twin Ion Engine Interceptor,1
2,TIE/sa bomber,1


In [34]:
#CTE
pd.read_sql(
    '''
        WITH CTE AS (
        SELECT 
            model, AVG(passengers) AS avg_pass
        FROM 
            vehicles
        GROUP BY
            model)
        SELECT
            model,
            MAX(avg_pass) AS Maximum_passengers
        FROM
            CTE
    ''', 
    conn
)



,model,Maximum_passengers
0,Modified Luxury Sail Barge,500.0
